<a href="https://colab.research.google.com/github/mingmingbupt/tensorflow/blob/master/tf_tfrecord_basic_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.2.0-rc2
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.2.1
numpy 1.18.2
pandas 1.0.3
sklearn 0.22.2.post1
tensorflow 2.2.0-rc2
tensorflow.keras 2.3.0-tf


In [3]:
# tfrecord 文件格式
# tfrecord其实是一个文件格式
# 它里面存储的内容呢，都是tf.trian.Example,这个Example呢可以是一个样本，也可以是一组样本
# 对于每一个example它里面是什么呢
# ---》里面都是一个一个的tf.train.Features.而对于Features呢，你可以把他理解成dict.在dict里面，
#   key就是你定义的feature的名字，value就是具体的值
#   而对于每个不同的tf.train.Features呢，他都有不同的格式，分别有tf.train.ByteList（存储字符串）/FloatList（存储float）/Int64List(存储整数)
#   这是tf.record的一个内容，他是一个层层往下分的过程
#   他把每个样本呢，都抽象成一系列的features,然后这些features组成一个example,把具体的example呢，按照某种方式存储到tf.record文件当中去，
#   就得到了tf.record的一个文件

# -> tf.train.Example
#    -> tf.train.Features -> {"key": tf.train.Feature}
#       -> tf.train.Feature -> tf.train.ByteList/FloatList/Int64List

#先看tf.train.ByteList/FloatList/Int64List，再看tf.train.Features，再看tf.train.Example，最后是tfrecord
favorite_books = [name.encode('utf-8')
                  for name in ["machine learning", "cc150"]] #需要把字符串转成utf8的格式，这里需要做一个转换
                  # 这样就得到要给utf8的列表
favorite_books_bytelist = tf.train.BytesList(value = favorite_books) # 再把这个列表传给tf.train.BytesList,来获得一个ByteList的对象，传的参数名字呢是value
print(favorite_books_bytelist)

hours_floatlist = tf.train.FloatList(value = [15.5, 9.5, 7.0, 8.0]) #同样的方法去初始化一个FloatList
print(hours_floatlist)

age_int64list = tf.train.Int64List(value = [42]) #同样的方法去初始化一个Int64List
print(age_int64list)

features = tf.train.Features( #在有了favorite_books hours_floatlist age_int64list以后我们就可以去创建features对象了
    # 我们使用tf.train.Features去构建featrues对象                        
    feature = { # 这里面需要传进去的是一个叫feature的参数，feature参数是一个dict
        "favorite_books": tf.train.Feature( #用favorite_books_bytelist去初始化一个tf.train.Feature对象
            bytes_list = favorite_books_bytelist),
        "hours": tf.train.Feature(
            float_list = hours_floatlist),
        "age": tf.train.Feature(int64_list = age_int64list),
    }
)
print(features) #打印features

value: "machine learning"
value: "cc150"

value: 15.5
value: 9.5
value: 7.0
value: 8.0

value: 42

feature {
  key: "age"
  value {
    int64_list {
      value: 42
    }
  }
}
feature {
  key: "favorite_books"
  value {
    bytes_list {
      value: "machine learning"
      value: "cc150"
    }
  }
}
feature {
  key: "hours"
  value {
    float_list {
      value: 15.5
      value: 9.5
      value: 7.0
      value: 8.0
    }
  }
}



In [4]:
example = tf.train.Example(features=features) #有了features 就可以组建一个example了
print(example)

serialized_example = example.SerializeToString() #可以把example序列化，因为存储的时候需要对内容进行一个压缩，以减少它的size
print(serialized_example)

features {
  feature {
    key: "age"
    value {
      int64_list {
        value: 42
      }
    }
  }
  feature {
    key: "favorite_books"
    value {
      bytes_list {
        value: "machine learning"
        value: "cc150"
      }
    }
  }
  feature {
    key: "hours"
    value {
      float_list {
        value: 15.5
        value: 9.5
        value: 7.0
        value: 8.0
      }
    }
  }
}

b'\n\\\n\x1d\n\x05hours\x12\x14\x12\x12\n\x10\x00\x00xA\x00\x00\x18A\x00\x00\xe0@\x00\x00\x00A\n-\n\x0efavorite_books\x12\x1b\n\x19\n\x10machine learning\n\x05cc150\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01*'


In [0]:
#如何把example存到文件当中去，生成一个具体的tfrecord的文件
output_dir = 'tfrecord_basic'
if not os.path.exists(output_dir):
    os.mkdir(output_dir) 
filename = "test.tfrecords"
filename_fullpath = os.path.join(output_dir, filename) #全路径
with tf.io.TFRecordWriter(filename_fullpath) as writer: #用tf.io.TFRecordWriter打开了一个tfrecord文件
    for i in range(3):
        writer.write(serialized_example) # 把刚才定义好的example写进去三次


In [9]:
#如何读取tfrecord文件呢
dataset = tf.data.TFRecordDataset([filename_fullpath])#传进去的是一个列表，是我们刚才定义的全路径，文件中写的都是序列化好的example，所以读出来
for serialized_example_tensor in dataset:
    print(serialized_example_tensor) #所以读出来的都是一堆字符串

tf.Tensor(b'\n\\\n\x1d\n\x05hours\x12\x14\x12\x12\n\x10\x00\x00xA\x00\x00\x18A\x00\x00\xe0@\x00\x00\x00A\n-\n\x0efavorite_books\x12\x1b\n\x19\n\x10machine learning\n\x05cc150\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01*', shape=(), dtype=string)
tf.Tensor(b'\n\\\n\x1d\n\x05hours\x12\x14\x12\x12\n\x10\x00\x00xA\x00\x00\x18A\x00\x00\xe0@\x00\x00\x00A\n-\n\x0efavorite_books\x12\x1b\n\x19\n\x10machine learning\n\x05cc150\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01*', shape=(), dtype=string)
tf.Tensor(b'\n\\\n\x1d\n\x05hours\x12\x14\x12\x12\n\x10\x00\x00xA\x00\x00\x18A\x00\x00\xe0@\x00\x00\x00A\n-\n\x0efavorite_books\x12\x1b\n\x19\n\x10machine learning\n\x05cc150\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01*', shape=(), dtype=string)


In [17]:
#如何把序列化后example解析成正常的肉眼可以识别的的example呢
#需要首先定义一个字典，字典中定义了每个feature及其所对应的类型
#这里有三个特征favorite_books hours age
expected_features = {
    "favorite_books": tf.io.VarLenFeature(dtype = tf.string), #类型是tf.io.VarLenFeature是一个变长的feature,具体的类型是字符串
    "hours": tf.io.VarLenFeature(dtype = tf.float32), #变长的feature,因为他是一个列表，他的类型是tf.float32
    "age": tf.io.FixedLenFeature([], dtype = tf.int64), #age是一个定长的类型，因为他只有一个元素，tf.io.FixedLenFeature，它的type是tf.int64
}#这样我们就定义好了我们features中三个数值的具体类型

dataset = tf.data.TFRecordDataset([filename_fullpath]) 
for serialized_example_tensor in dataset:
    #print(serialized_example_tensor) 读出来的都是一堆字符串，这里不再直接打印了，而是解析一下
    example = tf.io.parse_single_example( #调用的是tf.io.parse_single_example
        serialized_example_tensor,#具体的序列化后的tensor
        expected_features)    #期待的一个类型
    print("example=" , example)
    print("age=",example["age"])  
    print("favorite_books=",example["favorite_books"])      
    books = tf.sparse.to_dense(example["favorite_books"], #
                               default_value=b"") #不能把0当初默认值传成字符串，这里就把default_value改成一个默认字符串
    print("books=", books)
    for book in books:
        print(book.numpy().decode("UTF-8")) #之前做了encode 这里做一个decode

example= {'favorite_books': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f139a04b710>, 'hours': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f139a04b5c0>, 'age': <tf.Tensor: shape=(), dtype=int64, numpy=42>}
age= tf.Tensor(42, shape=(), dtype=int64)
favorite_books= SparseTensor(indices=tf.Tensor(
[[0]
 [1]], shape=(2, 1), dtype=int64), values=tf.Tensor([b'machine learning' b'cc150'], shape=(2,), dtype=string), dense_shape=tf.Tensor([2], shape=(1,), dtype=int64))
books= tf.Tensor([b'machine learning' b'cc150'], shape=(2,), dtype=string)
machine learning
cc150
example= {'favorite_books': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f139a04bf28>, 'hours': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f139a04b4e0>, 'age': <tf.Tensor: shape=(), dtype=int64, numpy=42>}
age= tf.Tensor(42, shape=(), dtype=int64)
favorite_books= SparseTensor(indices=tf.Tensor(
[[0]
 [1]], shape=(2, 1), dtype=int64), v

In [0]:
#除了正常存储 还可以把tfrecord存成压缩文件
filename_fullpath_zip = filename_fullpath + '.zip' #首先定义一个文件名，就是在正常文件后面加上.zip就可以了
options = tf.io.TFRecordOptions(compression_type = "GZIP") #我们如果想把文存成压缩形式，需要定义一个options，
#在compression_type里面定义一个compression_type，这里使用GZIP
#有了option以后再打开文件的时候，需要把option传进去，然后在进行存储
with tf.io.TFRecordWriter(filename_fullpath_zip, options) as writer:
    for i in range(3):
        writer.write(serialized_example)

In [20]:
#对于压缩后的文件，该如何读取呢
#读取的方法也很类似，在读取的时候，我们在创建dataset的时候，把这个compression_type也要传进去，这里他就等于GZIP
#其他部分不变
dataset_zip = tf.data.TFRecordDataset([filename_fullpath_zip],  compression_type= "GZIP")
for serialized_example_tensor in dataset_zip:
    example = tf.io.parse_single_example(
        serialized_example_tensor,
        expected_features)
    books = tf.sparse.to_dense(example["favorite_books"],
                               default_value=b"")
    for book in books:
        print(book.numpy().decode("UTF-8"))

machine learning
cc150
machine learning
cc150
machine learning
cc150
